In [1]:
import itertools
import functools
import os
import math

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Century Schoolbook']
rcParams['font.size'] = 13
rcParams['figure.dpi'] = 300
rcParams['text.usetex'] = False
from matplotlib.markers import CARETDOWNBASE, CARETUPBASE
import matplotlib.dates as mdates

import datetime as dt
from datetime import date, datetime, timedelta

import locale
locale.setlocale(locale.LC_ALL, 'it_IT')

from ETL_utilities import it_data_load, fix_2020_03_10, add_delta_growth_perc, add_ra, add_ra_days
from formatting_utilities import format_dates, format_dates_from_numbers, LegendTitle
from file_utilities import save_in_dir
from common_data import REGIONS_POP

%load_ext autoreload
%autoreload 2

# Global constants
# PLEASE CUSTOMIZE
OUTPUT_PATH = '../output'

In [2]:
# Caricamento dei dati dalla Protezione Civile
ds_it = it_data_load()

# Sanity check
ds_it.tail(10)

,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati,note_it,note_en
data,,,,,,,,,,,,,,,
2020-04-27,ITA,20353,1956,22309,83504,105813,-290,1739,66624,26977,199414,1789662,1237317.0,NaN,NaN
2020-04-28,ITA,19723,1863,21586,83619,105205,-608,2091,68941,27359,201505,1846934,1274871.0,NaN,NaN
2020-04-29,ITA,19210,1795,21005,83652,104657,-548,2086,71252,27682,203591,1910761,1313460.0,NaN,NaN
2020-04-30,ITA,18149,1694,19843,81708,101551,-3106,1872,75945,27967,205463,1979217,1354901.0,NaN,NaN
2020-05-01,ITA,17569,1578,19147,81796,100943,-608,1965,78249,28236,207428,2053425,1398633.0,dc-IT-0015,dc-EN-0015
2020-05-02,ITA,17357,1539,18896,81808,100704,-239,1900,79914,28710,209328,2108837,1429864.0,dc-IT-0017,dc-EN-0017
2020-05-03,ITA,17242,1501,18743,81436,100179,-525,1389,81654,28884,210717,2153772,1456911.0,NaN,NaN
2020-05-04,ITA,16823,1479,18302,81678,99980,-199,1221,82879,29079,211938,2191403,1479910.0,dc-IT-0019,dc-EN-0019
2020-05-05,ITA,16270,1427,17697,80770,98467,-1513,1075,85231,29315,213013,2246666,1512121.0,NaN,NaN


In [ ]:
ds_reg_last_day = ds_reg[last_day:].copy()
ds_reg_last_day['CFR'] = ds_reg_last_day['deceduti'] / ds_reg_last_day['totale_casi']
ds_reg_last_day['pop'] = ds_reg_last_day['denominazione_regione'].map(REGIONS_POP)
ds_reg_last_day['infection_density'] = ds_reg_last_day['totale_positivi'] / ds_reg_last_day['pop'] * 1_000
#ds_reg_last_day